In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
train = pd.read_csv('./data/Train_Fyxd0t8.csv')
test = pd.read_csv('./data/Test_C1XBIYq.csv')

In [4]:
train.head()

,ID,Estimated_Insects_Count,Crop_Type,Soil_Type,Pesticide_Use_Category,Number_Doses_Week,Number_Weeks_Used,Number_Weeks_Quit,Season,Crop_Damage
0,F00000001,188,1,0,1,0,0,0,1,0
1,F00000003,209,1,0,1,0,0,0,2,1
2,F00000004,257,1,0,1,0,0,0,2,1
3,F00000005,257,1,1,1,0,0,0,2,1
4,F00000006,342,1,0,1,0,0,0,2,1


In [5]:
train.Crop_Damage.value_counts()

0    74238
1    12307
2     2313
Name: Crop_Damage, dtype: int64

In [8]:
train = train.fillna(-1)
test = test.fillna(-1)

In [11]:
train = train.select_dtypes(exclude=['object'])
test = test.select_dtypes(exclude=['object'])

In [13]:
X = train[train.columns.drop('Crop_Damage')]
y = train.Crop_Damage

In [12]:
from sklearn.cross_validation import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=44)